In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

%matplotlib inline
from IPython import display


import functions.parse_data as parse
#import functions.handy_functions as hf
import torch.nn as nn
import torch


from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

#from tqdm import tqdm

import os

/tmp/ipykernel_14264/1751258556.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [14]:
channel_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06','Cloud_B07',
                 'Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13']

data_water=parse.parse('cloudrm2_water.dat')
data_clear=parse.parse('cloudrm2_clear.dat')
data_ice=parse.parse('cloudrm2_ice.dat')
data_mixed=parse.parse('cloudrm2_mixed.dat')

#Concatinate all datasets
#data_all=pd.concat([data_water, data_clear, data_ice, data_mixed])
data_all=pd.concat([data_water,data_clear,data_ice,data_mixed])
#data_all=pd.concat([data_ice])
#data_all=data_all.drop(columns=['Surface_Desc','Cloud_B01','Clear_B01'])
data_all=data_all.drop(columns=['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
                                'Clear_B07','Clear_B08','Clear_B09','Clear_B10','Clear_B11',
                                'Clear_B12','Clear_B13'])
data_all=data_all.reset_index()
data_all=data_all.drop(columns=('index'))

df=data_all.copy()

##Train test validation split##
#X_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
#           'Cloud_B07','Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13',
#           'Sat_Zenith_Angle','Sun_Zenith_Angle','Azimuth_Diff_Angle']

labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
           'Cloud_B07','Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13','COT']
#Leave out 'GOT', 'Water_Vapor'
#Band 1 no go.

In [20]:
df['Surface_Desc']=df.Surface_Desc.astype('category')
df['Surface_Desc']=pd.factorize(df.Surface_Desc)[0]

In [16]:
X=df[labels]

X_train, X_test=train_test_split(df,test_size=0.2)
X_test,X_val=train_test_split(X_test,test_size=0.5)


In [17]:
X_train.to_csv("cot_train/data/synthetic-cot-data/train")
X_test.to_csv("cot_train/data/synthetic-cot-data/test")
X_val.to_csv("cot_train/data/synthetic-cot-data/val")

X_train_npy=X_train.to_numpy()
X_test_npy=X_test.to_numpy()
X_val_npy=X_val.to_numpy()

np.save("cot_train/data/synthetic-cot-data/trainset_smhi.npy",X_train_npy,allow_pickle=True)
np.save("cot_train/data/synthetic-cot-data/testset_smhi",X_test_npy,allow_pickle=True)
np.save("cot_train/data/synthetic-cot-data/valset_smhi",X_val_npy,allow_pickle=True)



## Load model and test data

In [4]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)

CUDA is available. Using GPU.
Tensor is on device: cuda:0


In [7]:
from cot_train.utils import StatCollector, MLP5

In [12]:
tmp=np.load('cot_train/log/2024-03-11_13-48-42/train_stats/MSE_loss_test.npz')


NpzFile 'cot_train/log/2024-03-11_13-48-42/train_stats/MSE_loss_test.npz' with keys: values, means, mas, times

In [16]:
with np.load('cot_train/log/2024-03-11_13-48-42/train_stats/MSE_loss_test.npz') as data:
    means=data['means']
    vals=data['values']
    mas=data['mas']
    times=data['times']

In [23]:
times

array([   1,    2,    3, ..., 3719, 3720, 3721])